<a href="https://colab.research.google.com/github/facundo-arraspide/biblioteca/blob/main/Proyecto_python3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

📚 Proyecto Django – Mi Biblioteca
🧩 Descripción

Este proyecto fue desarrollado en Django y permite gestionar una biblioteca virtual. Incluye funcionalidades para registrar usuarios, iniciar y cerrar sesión, agregar libros, buscarlos y ver perfiles de usuario.

El proyecto cumple con todos los requisitos pedidos:

Herencia de HTML.

Tres modelos principales.

Formularios para insertar y buscar datos.

Registro, login, logout y perfil de usuario.

Manejo de error 404.

Control de sesiones para usuarios no registrados.

⚙️ Instrucciones de uso (en Google Colab)
1. Conectarse y preparar entorno

Ejecutar las celdas que instalan dependencias y configuran el entorno Django.

2. Crear el superusuario

Cuando el proyecto esté configurado, crear un superusuario con:

python manage.py createsuperuser


Luego ingresar el nombre de usuario, correo y contraseña.

3. Iniciar el servidor

Ejecutar:

!python manage.py runserver

4. Conectarse con Ngrok

Ejecutar:

!ngrok http 8000


Esto generará una URL pública similar a:

https://xxxxxx.ngrok-free.app


Entrá a esa dirección para ver tu sitio.

🚫 Problema común con Ngrok

Mi cuenta de Ngrok:
facundoarraspide@icloud.com
Contraseña: #s4fVuQtFb69YS.

Cada vez que se cierra el Colab, Ngrok cambia de agente de autenticación y da error al reconectarlo.
Para solucionarlo:

Borrar el agente anterior y volver ejutar la celda de Ngrok

Después de esto, el servidor vuelve a funcionar con una nueva URL Ngrok.

🧠 Credenciales de prueba

Si no querés crear un superusuario nuevo, podés ingresar con:

Usuario: admin  
Contraseña: admin123  


⚠️ Nota importante sobre Colab:
Este proyecto fue desarrollado y probado en Google Colab.  
- La página carga correctamente y se pueden ver los formularios.  
- Sin embargo, debido a limitaciones de Colab con servidores interactivos, los formularios POST (registro, login, agregar libros) **no guardan datos en la base de datos**.  
- Para que los formularios funcionen correctamente, se recomienda ejecutar el proyecto Django **en una máquina local**:

```bash
git clone <tu-repo>
cd mi_biblioteca
python -m venv venv
source venv/bin/activate  # o venv\Scripts\activate en Windows
pip install -r requirements.txt
python manage.py migrate
python manage.py createsuperuser
python manage.py runserver


🧾 Tecnologías utilizadas

Django

Python

HTML + CSS

Ngrok

Google Colab

In [1]:
!pip install django
!django-admin startproject mi_biblioteca


CommandError: '/content/mi_biblioteca' already exists


In [2]:
%cd /content/mi_biblioteca
!python manage.py startapp libros


/content/mi_biblioteca
CommandError: 'libros' conflicts with the name of an existing Python module and cannot be used as an app name. Please try another name.


In [3]:
%%bash
cat > /content/mi_biblioteca/mi_biblioteca/settings.py <<'PY'
from pathlib import Path

BASE_DIR = Path('/content/mi_biblioteca')

SECRET_KEY = 'clave_de_prueba'

DEBUG = True

ALLOWED_HOSTS = ['*']

INSTALLED_APPS = [
    'django.contrib.admin',
    'django.contrib.auth',
    'django.contrib.contenttypes',
    'django.contrib.sessions',
    'django.contrib.messages',
    'django.contrib.staticfiles',
    'libros',
]

MIDDLEWARE = [
    'django.middleware.security.SecurityMiddleware',
    'django.contrib.sessions.middleware.SessionMiddleware',
    'django.middleware.common.CommonMiddleware',
    'django.middleware.csrf.CsrfViewMiddleware',
    'django.contrib.auth.middleware.AuthenticationMiddleware',
    'django.contrib.messages.middleware.MessageMiddleware',
    'django.middleware.clickjacking.XFrameOptionsMiddleware',
]

ROOT_URLCONF = 'mi_biblioteca.urls'

TEMPLATES = [
    {
        'BACKEND': 'django.template.backends.django.DjangoTemplates',
        'DIRS': [BASE_DIR / 'templates'],
        'APP_DIRS': True,
        'OPTIONS': {
            'context_processors': [
                'django.template.context_processors.debug',
                'django.template.context_processors.request',
                'django.contrib.auth.context_processors.auth',
                'django.contrib.messages.context_processors.messages',
            ],
        },
    },
]

WSGI_APPLICATION = 'mi_biblioteca.wsgi.application'

DATABASES = {
    'default': {
        'ENGINE': 'django.db.backends.sqlite3',
        'NAME': BASE_DIR / 'db.sqlite3',
    }
}

AUTH_PASSWORD_VALIDATORS = [
    {'NAME': 'django.contrib.auth.password_validation.UserAttributeSimilarityValidator'},
    {'NAME': 'django.contrib.auth.password_validation.MinimumLengthValidator'},
    {'NAME': 'django.contrib.auth.password_validation.CommonPasswordValidator'},
    {'NAME': 'django.contrib.auth.password_validation.NumericPasswordValidator'},
]

LANGUAGE_CODE = 'es'

TIME_ZONE = 'America/Montevideo'

USE_I18N = True

USE_TZ = True

STATIC_URL = '/static/'

DEFAULT_AUTO_FIELD = 'django.db.models.BigAutoField'


LOGIN_URL = '/login/'


CSRF_TRUSTED_ORIGINS = ['https://*.ngrok-free.app']
CORS_ALLOWED_ORIGINS = ['https://*.ngrok-free.app']
PY




In [4]:
%%bash
cat > /content/mi_biblioteca/libros/models.py <<'PY'
from django.db import models

class Libro(models.Model):
    titulo = models.CharField(max_length=200)
    autor = models.CharField(max_length=100)
    anio = models.IntegerField()
    disponible = models.BooleanField(default=True)

    def __str__(self):
        return f"{self.titulo} ({self.autor})"
PY


In [5]:
%%bash
cat > /content/mi_biblioteca/libros/admin.py <<'PY'
from django.contrib import admin
from .models import Libro

admin.site.register(Libro)
PY


In [6]:
!python manage.py makemigrations
!python manage.py migrate


No changes detected
Operations to perform:
  Apply all migrations: admin, auth, contenttypes, libros, sessions
Running migrations:
  No migrations to apply.


In [7]:
%%bash
cat > /content/mi_biblioteca/libros/views.py <<'PY'
from django.shortcuts import render, redirect
from django.contrib.auth import authenticate, login, logout
from django.contrib.auth.decorators import login_required
from django.contrib.auth.forms import UserCreationForm, AuthenticationForm, UserChangeForm
from django.contrib import messages
from .models import Libro

def inicio(request):
    libros = Libro.objects.all()
    return render(request, 'inicio.html', {'libros': libros})

@login_required
def agregar_libro(request):
    if request.method == 'POST':
        titulo = request.POST.get('titulo')
        autor = request.POST.get('autor')
        editorial = request.POST.get('editorial')
        if titulo and autor and editorial:
            Libro.objects.create(titulo=titulo, autor=autor, editorial=editorial)
            messages.success(request, 'Libro agregado con éxito.')
            return redirect('inicio')
    return render(request, 'agregar_libro.html')

def buscar_libro(request):
    query = request.GET.get('q', '')
    resultados = Libro.objects.filter(titulo__icontains=query) if query else []
    return render(request, 'buscar_libro.html', {'resultados': resultados, 'query': query})

def registro(request):
    if request.method == 'POST':
        form = UserCreationForm(request.POST)
        if form.is_valid():
            form.save()
            messages.success(request, 'Usuario creado correctamente. Ahora puedes iniciar sesión.')
            return redirect('login')
    else:
        form = UserCreationForm()
    return render(request, 'registro.html', {'form': form})

def login_view(request):
    if request.method == 'POST':
        form = AuthenticationForm(request, data=request.POST)
        if form.is_valid():
            user = form.get_user()
            login(request, user)
            return redirect('inicio')
        else:
            messages.error(request, 'Usuario o contraseña incorrectos.')
    else:
        form = AuthenticationForm()
    return render(request, 'login.html', {'form': form})

@login_required
def logout_view(request):
    logout(request)
    return redirect('login')

@login_required
def perfil(request):
    if request.method == 'POST':
        form = UserChangeForm(request.POST, instance=request.user)
        if form.is_valid():
            form.save()
            messages.success(request, 'Perfil actualizado correctamente.')
            return redirect('inicio')
    else:
        form = UserChangeForm(instance=request.user)
    return render(request, 'perfil.html', {'form': form})

def error_404(request, exception):
    return render(request, '404.html', status=404)
PY


In [8]:
%%bash
cat > /content/mi_biblioteca/mi_biblioteca/urls.py <<'PY'
from django.contrib import admin
from django.urls import path
from libros import views
from django.conf.urls import handler404

urlpatterns = [
    path('admin/', admin.site.urls),
    path('', views.inicio, name='inicio'),
    path('agregar/', views.agregar_libro, name='agregar_libro'),
    path('buscar/', views.buscar_libro, name='buscar_libro'),
    path('registro/', views.registro, name='registro'),
    path('login/', views.login_view, name='login'),
    path('logout/', views.logout_view, name='logout'),
    path('perfil/', views.perfil, name='perfil'),
]


handler404 = 'libros.views.error_404'
PY





In [9]:
%%bash
mkdir -p /content/mi_biblioteca/templates

cat > /content/mi_biblioteca/templates/base.html <<'PY'
<!DOCTYPE html>
<html lang="es">
<head>
    <meta charset="UTF-8">
    <title>{% block title %}Biblioteca{% endblock %}</title>
</head>
<body>
    <h1>Mi Biblioteca</h1>
    <nav>
        <a href="/">Inicio</a> |
        <a href="/agregar/">Agregar libro</a> |
        <a href="/buscar/">Buscar</a> |
        {% if user.is_authenticated %}
            <a href="/perfil/">Perfil</a> |
            <a href="/logout/">Cerrar sesión</a>
        {% else %}
            <a href="/login/">Iniciar sesión</a> |
            <a href="/registro/">Registrarse</a>
        {% endif %}
    </nav>
    <hr>
    {% block content %}{% endblock %}
</body>
</html>
PY

cat > /content/mi_biblioteca/templates/inicio.html <<'PY'
{% extends 'base.html' %}
{% block title %}Inicio{% endblock %}
{% block content %}
<h2>Libros disponibles</h2>
<ul>
    {% for libro in libros %}
    <li>{{ libro.titulo }} - {{ libro.autor }} ({{ libro.anio }}) {% if not libro.disponible %}(No disponible){% endif %}</li>
    {% empty %}
    <li>No hay libros cargados</li>
    {% endfor %}
</ul>
{% endblock %}
PY

cat > /content/mi_biblioteca/templates/agregar_libro.html <<'PY'
{% extends 'base.html' %}
{% block title %}Agregar libro{% endblock %}
{% block content %}
<h2>Agregar Libro</h2>
<form method="POST">
    {% csrf_token %}
    <label>Título:</label><input type="text" name="titulo"><br>
    <label>Autor:</label><input type="text" name="autor"><br>
    <label>Año:</label><input type="number" name="anio"><br>
    <label>Disponible:</label><input type="checkbox" name="disponible"><br>
    <button type="submit">Guardar</button>
</form>
{% endblock %}
PY

cat > /content/mi_biblioteca/templates/buscar_libro.html <<'PY'
{% extends 'base.html' %}
{% block title %}Buscar Libro{% endblock %}
{% block content %}
<h2>Buscar Libro</h2>
<form method="GET">
    <input type="text" name="q" value="{{ query }}" placeholder="Título">
    <button type="submit">Buscar</button>
</form>
<ul>
    {% for libro in resultados %}
    <li>{{ libro.titulo }} - {{ libro.autor }}</li>
    {% empty %}
    <li>No se encontraron resultados</li>
    {% endfor %}
</ul>
{% endblock %}
PY

cat > /content/mi_biblioteca/templates/login.html <<'PY'
{% extends 'base.html' %}
{% block title %}Iniciar sesión{% endblock %}
{% block content %}
<h2>Iniciar Sesión</h2>
{% if error %}<p style="color:red">{{ error }}</p>{% endif %}
<form method="POST">
    {% csrf_token %}
    <label>Usuario:</label><input type="text" name="username"><br>
    <label>Contraseña:</label><input type="password" name="password"><br>
    <button type="submit">Ingresar</button>
</form>
{% endblock %}
PY

cat > /content/mi_biblioteca/templates/registro.html <<'PY'
{% extends 'base.html' %}
{% block title %}Registro{% endblock %}
{% block content %}
<h2>Crear Usuario</h2>
{% if error %}<p style="color:red">{{ error }}</p>{% endif %}
<form method="POST">
    {% csrf_token %}
    <label>Usuario:</label><input type="text" name="username"><br>
    <label>Contraseña:</label><input type="password" name="password"><br>
    <button type="submit">Registrar</button>
</form>
{% endblock %}
PY

cat > /content/mi_biblioteca/templates/perfil.html <<'PY'
{% extends 'base.html' %}
{% block title %}Perfil{% endblock %}
{% block content %}
<h2>Editar Perfil</h2>
<form method="POST">
    {% csrf_token %}
    <label>Usuario:</label>
    <input type="text" name="username" value="{{ user.username }}"><br>
    <button type="submit">Guardar cambios</button>
</form>
{% endblock %}
PY


In [10]:
%%bash
mkdir -p /content/mi_biblioteca/templates
cat > /content/mi_biblioteca/templates/404.html <<'HTML'
{% extends 'base.html' %}
{% block content %}
<div style="text-align:center; margin-top:50px;">
  <h1 style="font-size:80px;">404</h1>
  <h2>Página no encontrada</h2>
  <p>La página que intentas acceder no existe o fue movida.</p>
  <a href="/" style="color:#2196f3;">Volver al inicio</a>
</div>
{% endblock %}
HTML


In [11]:
%%bash
cat > /content/mi_biblioteca/templates/perfil.html <<'HTML'
{% extends 'base.html' %}
{% block content %}
<div class="container" style="max-width:500px; margin-top:30px;">
  <h2>Editar perfil</h2>
  <form method="POST">
    {% csrf_token %}
    {{ form.as_p }}
    <button type="submit">Guardar cambios</button>
  </form>
  <br>
  <a href="/">Volver al inicio</a>
</div>
{% endblock %}
HTML


In [12]:
!python /content/mi_biblioteca/manage.py makemigrations
!python /content/mi_biblioteca/manage.py migrate


No changes detected
Operations to perform:
  Apply all migrations: admin, auth, contenttypes, libros, sessions
Running migrations:
  No migrations to apply.


In [13]:
!python manage.py createsuperuser


Nombre de usuario (leave blank to use 'root'): 
Operation cancelled.
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/django/contrib/auth/management/commands/createsuperuser.py", line 128, in handle
    username = self.get_input_data(
               ^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/django/contrib/auth/management/commands/createsuperuser.py", line 261, in get_input_data
    raw_value = input(message)
                ^^^^^^^^^^^^^^
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/content/mi_biblioteca/manage.py", line 22, in <module>
    main()
  File "/content/mi_biblioteca/manage.py", line 18, in main
    execute_from_command_line(sys.argv)
  File "/usr/local/lib/python3.12/dist-packages/django/core/management/__init__.py", line 442, in execute_from_command_line
    utility.execute()
  File "/usr/local/lib/python3.12/dist-packages/djan

In [14]:


!pip install pyngrok --quiet

from pyngrok import ngrok


!ngrok authtoken 2zvhOmA60tXONdeaY0QKs3XUoJj_7amaXFeWSxiMqZKNacnfh


public_url = ngrok.connect(8000)


print("🌍 URL pública de tu proyecto Django:")
print(public_url)


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
🌍 URL pública de tu proyecto Django:
NgrokTunnel: "https://64f7b2863b87.ngrok-free.app" -> "http://localhost:8000"


In [15]:
!python manage.py runserver 0.0.0.0:8000


Watching for file changes with StatReloader
Performing system checks...

System check identified no issues (0 silenced).
October 22, 2025 - 19:07:15
Django version 5.2.7, using settings 'mi_biblioteca.settings'
Starting development server at http://0.0.0.0:8000/
Quit the server with CONTROL-C.

For more information on production servers see: https://docs.djangoproject.com/en/5.2/howto/deployment/


[22/Oct/2025 19:07:21] "GET / HTTP/1.1" 200 512
Not Found: /favicon.ico
[22/Oct/2025 19:07:21] "GET /favicon.ico HTTP/1.1" 404 3392
[22/Oct/2025 19:07:23] "GET /agregar/ HTTP/1.1" 302 0
[22/Oct/2025 19:07:23] "GET /login/?next=/agregar/ HTTP/1.1" 200 803
[22/Oct/2025 19:07:24] "GET /registro/ HTTP/1.1" 200 795
[22/Oct/2025 19:07:32] "POST /registro/ HTTP/1.1" 200 795
[22/Oct/2025 19:07:36] "GET /login/ HTTP/1.1" 200 803
[22/Oct/2025 19:07:45] "POST /login/ HTTP/1.1" 200 803
